What do I need for LIFO?
Say we buy some orders. Make them a stack.
[(Price1, Order1), (Price2, Order2), (Price3, Order3), (Price4, Order4) ... (PriceN, OrderN)]
A cover order appears.
Cover orders start appearing. 
Each cover order can be used to pop the buy order stack. 

In [4]:
def calculate_lifo_pnl(buy_stack, sell_quantity, sell_price):
    """
    Calculate the realized PnL using LIFO method.

    :param buy_stack: List of tuples [(buy_price1, quantity1), (buy_price2, quantity2), ...]
    :param sell_quantity: Quantity of the sell order
    :param sell_price: Price of the sell order
    :return: Realized PnL
    """
    realized_pnl = 0.0
    remaining_sell_quantity = sell_quantity
    counter = 0
    while remaining_sell_quantity > 0 and buy_stack:
        # Get the last buy order
        buy_price, buy_quantity = buy_stack.pop()

        # Determine the quantity to match
        match_quantity = min(buy_quantity, remaining_sell_quantity)


        # Calculate PnL for the matched quantity
        realized_pnl += match_quantity * (sell_price - buy_price) * 16 * 62.5

        if counter == 0:
            if match_quantity < buy_quantity:
                LIFO_complement = buy_quantity - match_quantity
                LIFO_complement_pnl = LIFO_complement * (sell_price - buy_price) * 16 * 62.5
                return realized_pnl, LIFO_complement_pnl

        # Update the remaining sell quantity
        remaining_sell_quantity -= match_quantity

        # If there is leftover buy quantity, push it back to the stack
        if buy_quantity > match_quantity:
            buy_stack.append((buy_price, buy_quantity - match_quantity))
        counter += 1
    # If there are not enough buy orders to match the sell quantity, raise an error
    if remaining_sell_quantity > 0:
        raise ValueError("Not enough buy orders to match the sell quantity.")

    return realized_pnl, None

# Example usage
buy_orders = [(100, 10), (105, 5), (110, 15)]  # Example buy stack
sell_quantity = 20
sell_price = 115

pnl = calculate_lifo_pnl(buy_orders, sell_quantity, sell_price)
print(f"Realized PnL: {pnl}")

Realized PnL: 125000.0


In [1]:
from typing import List, Tuple

def lifo_pnl(trades: List[Tuple[int, float]]):
    """
    LIFO real-time PnL calculator.
    trades: list of (quantity, price); qty>0 = buy, qty<0 = sell.
    Returns: (realized_pnl_per_trade, open_lots, cumulative_pnl)
    """
    stack: List[Tuple[int, float]] = []      # open lots, LIFO
    realized_series, cum_pnl = [], 0.0

    for qty, price in trades:
        remaining, pnl_this = qty, 0.0

        # 1️⃣ offset against opposite lots
        while remaining and stack and ((remaining > 0) ^ (stack[-1][0] > 0)):
            lot_qty, lot_price = stack.pop()
            match = min(abs(remaining), abs(lot_qty))

            if remaining < 0:        # selling long
                pnl_this += (price - lot_price) * match
            else:                    # buying to cover short
                pnl_this += (lot_price - price) * match

            # shrink lot & remaining
            if abs(lot_qty) > match:
                stack.append((lot_qty // abs(lot_qty) * (abs(lot_qty) - match), lot_price))
            remaining = remaining // abs(remaining) * (abs(remaining) - match)

        # 2️⃣ leftover becomes a new open lot
        if remaining:
            stack.append((remaining, price))

        cum_pnl += pnl_this
        realized_series.append((pnl_this, cum_pnl))

    return realized_series, stack, cum_pnl


# ── quick demo ──────────────────────────────────────────────────────────────
trades = [
    (10, 100),   # buy 10 @100
    (20, 101),   # buy 20 @101
    (30, 102),   # buy 30 @102
    (-45, 103),  # sell 45 @103  → +60 PnL
    (-40, 104),  # sell 40 @104  → +55 PnL (goes short 25)
    (10, 103),   # buy 10 @103   → +10 PnL (still short 15)
    (20, 105)    # buy 20 @105   → –15 PnL (flip long 5)
]

realized, open_lots, total = lifo_pnl(trades)
for i, (pnl, cum) in enumerate(realized, 1):
    print(f"Trade {i:2}: ΔPnL = {pnl:6.1f}, Cumulative = {cum:6.1f}")
print("Remaining open lots (qty, price):", open_lots)


Trade  1: ΔPnL =    0.0, Cumulative =    0.0
Trade  2: ΔPnL =    0.0, Cumulative =    0.0
Trade  3: ΔPnL =    0.0, Cumulative =    0.0
Trade  4: ΔPnL =   60.0, Cumulative =   60.0
Trade  5: ΔPnL =   55.0, Cumulative =  115.0
Trade  6: ΔPnL =   10.0, Cumulative =  125.0
Trade  7: ΔPnL =  -15.0, Cumulative =  110.0
Remaining open lots (qty, price): [(5, 105)]


: 

In [8]:
import pickle
current_stack = pickle.load(open("lifo_streaming_state.pkl", "rb"))

{'stack': [(-13.0, 111.765625),
  (-987.0, 111.75),
  (-18.0, 111.25),
  (-727.0, 111.21875),
  (-39.0, 111.21875)],
 'last_processed_row': 286,
 'processed_transactions': {'2025-06-27 16:00:14.073000_1_10.0_111.796875_b94c03d5-257a-4bba-b053-84e72a7025a4',
  '2025-06-27 16:08:24.583000_1_5.0_111.84375_f618faee-877a-4e57-a71f-586bb613c5d2',
  '2025-06-27 16:08:24.584000_1_2.0_111.84375_f618faee-877a-4e57-a71f-586bb613c5d2',
  '2025-06-27 16:08:24.584000_1_3.0_111.84375_f618faee-877a-4e57-a71f-586bb613c5d2',
  '2025-06-27 16:13:09.813000_1_10.0_111.875_654a3b6e-ce05-4138-8793-732f04db44e4',
  '2025-06-30 10:20:40.340000_1_10.0_111.84375_e4ce12da-9930-4975-8688-5f29f52dd0e8',
  '2025-06-30 10:27:35.526000_1_1.0_111.890625_0cc5196e-6f9a-4fdb-84fa-c56f1389fbae',
  '2025-06-30 10:27:42.926000_1_5.0_111.890625_4090e33f-1ee6-49cf-97a0-d942ea2ebcc0',
  '2025-06-30 10:29:16.298000_2_1.0_111.859375_92deb39b-678c-4411-b606-a201740489eb',
  '2025-07-01 10:40:41.767000_1_45.0_111.828125_adee326d-1a

In [10]:
import pandas as pd
df = pd.read_csv("C:/Users/YamanSanghavi/Desktop/scenario_ladder_standalone/data/output/ladder/continuous_fills.csv")
df[df['SideName'] == "BUY"]['Quantity'].sum() - df[df['SideName'] == "SELL"]['Quantity'].sum()

np.float64(-147.0)

In [12]:
import pandas as pd
df = pd.read_csv("filled_orders_copiedTT.csv")
df

,3-Jul-25,27:01.6,CME,ZN Sep25,B,1,111'075,F,TT SIM,Trading,Eric,Eric.1,33f120f9-1c34-4508-a651-7a8172775e42,Unnamed: 13
0,3-Jul-25,41:57.3,CME,ZN Sep25,B,2,111'085,F,TT SIM,Trading,Eric,Eric,f5368272-922e-4bee-8197-1ff0e433b2fb,NaN
1,3-Jul-25,41:00.0,CME,ZN Sep25,B,1,111'085,F,TT SIM,Trading,Eric,Eric,1ce3510b-4347-4707-8b52-e283b11b9e13,NaN
2,3-Jul-25,56:47.1,CME,ZN Sep25,B,1,111'070,F,TT SIM,Trading,Eric,Eric,ee8254a5-a339-482d-ac0a-f129ecc1ef24,NaN
3,3-Jul-25,47:13.6,CME,ZN Sep25,B,1,111'070,F,TT SIM,Trading,Eric,Eric,b80bbe1b-f7d7-434a-a432-2779d6a7e06a,NaN
4,3-Jul-25,10:46.4,CME,ZN Sep25,B,1,111'065,F,TT SIM,Trading,Eric,Eric,2d1171d9-dd9a-475b-ac47-c719ef37476d,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,7-May-25,22:17.6,CME_Delayed,ZN Jun25,S,15,111'070,F,TT SIM,Trading,Eric,Eric,df6d779c-37c7-4568-af09-b45b686ac654,acfba931-9779-49e6-9c9d-3785cc19c61c
271,7-May-25,14:52.3,CME_Delayed,ZN Jun25,B,20,111'050,F,TT SIM,Trading,Eric,Eric,53121e79-aad5-480e-8f63-64a75365b15b,acfba931-9779-49e6-9c9d-3785cc19c61c
272,6-May-25,17:50.3,CME_Delayed,ZN Jun25,S,4,111'085,F,TT SIM,Trading,Eric,Eric,43b95a0c-a848-4d5c-aa65-8913b389c594,acfba931-9779-49e6-9c9d-3785cc19c61c
273,6-May-25,54:16.5,CME_Delayed,ZN Jun25,B,10,111'065,F,TT SIM,Trading,Eric,Eric,ed204150-cc58-41c2-9722-86d68dc2522f,acfba931-9779-49e6-9c9d-3785cc19c61c


In [23]:
df = pd.read_csv("AuditTrail.csv")
df = df[df["Exch"]=="CME"]
df = df[df["FillQty"]>0]
df

,Time,Exch,Contract,Message Type,ExecType,B/S,OrdQty,FillQty,WorkQty,ExeQty,...,AlgoName,TT Latency,Exch Latency,Route,Originator,CurrentUser,TTOrder ID,Parent ID,ExchOrderID,Date
1,27:24.3,CME,ZN Sep25,ExecutionReport,Trade,S,2000.0,1.0,1228.0,772.0,...,NaN,NaN,NaN,TT SIM,Eric,Eric,e26cf880-c539-434f-ac5a-263372fedb87,NaN,NaN,3-Jul-25
2,27:24.2,CME,ZN Sep25,ExecutionReport,Trade,S,2000.0,44.0,1229.0,771.0,...,NaN,NaN,NaN,TT SIM,Eric,Eric,e26cf880-c539-434f-ac5a-263372fedb87,NaN,NaN,3-Jul-25
3,27:22.7,CME,ZN Sep25,ExecutionReport,Trade,S,2000.0,727.0,1273.0,727.0,...,NaN,NaN,NaN,TT SIM,Eric,Eric,e26cf880-c539-434f-ac5a-263372fedb87,NaN,NaN,3-Jul-25
7,48:36.5,CME,ZN Sep25,ExecutionReport,Trade,B,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,TT SIM,Eric,Eric,e4716b30-fc6d-4269-8d94-7e99a6c7faa0,NaN,NaN,3-Jul-25
13,27:01.6,CME,ZN Sep25,ExecutionReport,Trade,B,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,TT SIM,Eric,Eric,33f120f9-1c34-4508-a651-7a8172775e42,NaN,NaN,3-Jul-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,13:09.8,CME,ZN Sep25,ExecutionReport,Trade,B,10.0,10.0,NaN,10.0,...,rough,NaN,NaN,TT SIM,Eric,Eric,654a3b6e-ce05-4138-8793-732f04db44e4,cec72a34-cf2b-4b24-894e-998b9ba5b3db,NaN,27-Jun-25
457,08:24.6,CME,ZN Sep25,ExecutionReport,Trade,B,10.0,3.0,NaN,10.0,...,rough,NaN,NaN,TT SIM,Eric,Eric,f618faee-877a-4e57-a71f-586bb613c5d2,5fffe889-ae31-4c5f-b60c-094da2d946d3,NaN,27-Jun-25
458,08:24.6,CME,ZN Sep25,ExecutionReport,Trade,B,10.0,2.0,3.0,7.0,...,rough,NaN,NaN,TT SIM,Eric,Eric,f618faee-877a-4e57-a71f-586bb613c5d2,5fffe889-ae31-4c5f-b60c-094da2d946d3,NaN,27-Jun-25
459,08:24.6,CME,ZN Sep25,ExecutionReport,Trade,B,10.0,5.0,5.0,5.0,...,rough,NaN,NaN,TT SIM,Eric,Eric,f618faee-877a-4e57-a71f-586bb613c5d2,5fffe889-ae31-4c5f-b60c-094da2d946d3,NaN,27-Jun-25


In [25]:
df[df['B/S'] == "B"]['FillQty'].sum() - df[df['B/S'] == "S"]['FillQty'].sum()

np.float64(-1790.0)

In [3]:
# Cell 9: Calculate net position for trades after June 27, 2025 00:00:00
import pandas as pd
from datetime import datetime

# Read the continuous fills data
df = pd.read_csv("data/output/ladder/continuous_fills.csv")
df = df[df["Exchange"]=="CME"]
df = df[df["CurrentUser"]=="Eric"]
df = df[df["Contract"]=="ZN Sep25"]
df[df["SideName"]=="BUY"]["Quantity"].sum() - df[df["SideName"]=="SELL"]["Quantity"].sum()

np.float64(-1784.0)

In [27]:
# Cell 9: Calculate net position for trades after June 27, 2025 00:00:00
import pandas as pd
from datetime import datetime

# Read the continuous fills data
df = pd.read_csv("data/output/ladder/continuous_fills.csv")

# Create a proper datetime column by combining Date and Time
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Filter for trades after June 27, 2025 00:00:00
cutoff_date = datetime(2025, 6, 27, 0, 0, 0)
filtered_df = df[df['DateTime'] <= cutoff_date]

print(f"Total trades in dataset: {len(df)}")
print(f"Trades after June 27, 2025 00:00:00: {len(filtered_df)}")
print(f"Date range of filtered data: {filtered_df['DateTime'].min()} to {filtered_df['DateTime'].max()}")

# Calculate net position (Buy quantities - Sell quantities)
buy_quantity = filtered_df[filtered_df['SideName'] == 'BUY']['Quantity'].sum()
sell_quantity = filtered_df[filtered_df['SideName'] == 'SELL']['Quantity'].sum()
net_position = buy_quantity - sell_quantity

print(f"\n=== NET POSITION CALCULATION AFTER JUNE 27, 2025 ===")
print(f"Total BUY quantity: {buy_quantity}")
print(f"Total SELL quantity: {sell_quantity}")
print(f"Net Position (Buy - Sell): {net_position}")

# Show breakdown by instrument if multiple instruments exist
print(f"\n=== BREAKDOWN BY INSTRUMENT ===")
instrument_summary = filtered_df.groupby('InstrumentId').agg({
    'Quantity': lambda x: filtered_df.loc[x.index][filtered_df.loc[x.index]['SideName'] == 'BUY']['Quantity'].sum() - 
                          filtered_df.loc[x.index][filtered_df.loc[x.index]['SideName'] == 'SELL']['Quantity'].sum()
}).rename(columns={'Quantity': 'Net_Position'})

for instrument_id, row in instrument_summary.iterrows():
    instrument_name = filtered_df[filtered_df['InstrumentId'] == instrument_id]['InstrumentName'].iloc[0]
    print(f"Instrument {instrument_id} ({instrument_name}): {row['Net_Position']}")

# Show the actual trades after June 27, 2025
print(f"\n=== TRADES AFTER JUNE 27, 2025 ===")
display_df = filtered_df[['DateTime', 'SideName', 'Quantity', 'Price', 'InstrumentName']].copy()
display_df = display_df.sort_values('DateTime')
print(display_df.to_string(index=False))

Total trades in dataset: 280
Trades after June 27, 2025 00:00:00: 217
Date range of filtered data: 2025-05-06 14:50:49.648000 to 2025-06-04 13:26:03.785000

=== NET POSITION CALCULATION AFTER JUNE 27, 2025 ===
Total BUY quantity: 1681.0
Total SELL quantity: 809.0
Net Position (Buy - Sell): 872.0

=== BREAKDOWN BY INSTRUMENT ===
Instrument 1733802083425552218 (Unknown_1733802083425552218): -5.0
Instrument 8092262517946017060 (Unknown_8092262517946017060): 822.0
Instrument 14868446001769503902 (Unknown_14868446001769503902): 55.0

=== TRADES AFTER JUNE 27, 2025 ===
               DateTime SideName  Quantity         Price               InstrumentName
2025-05-06 14:50:49.648      BUY       1.0    111.281250  Unknown_8092262517946017060
2025-05-06 19:54:16.538      BUY      10.0    111.203125  Unknown_8092262517946017060
2025-05-06 20:17:50.331     SELL       4.0    111.265625  Unknown_8092262517946017060
2025-05-07 04:14:52.340      BUY      20.0    111.156250  Unknown_8092262517946017060


In [28]:
filtered_df

,Date,Time,InstrumentId,InstrumentName,Side,SideName,Quantity,Price,OrderId,AccountId,MarketId,TransactTime,TimeStamp,ExecId,OrderStatus,DateTime
0,2025-05-06,14:50:49.648,8092262517946017060,Unknown_8092262517946017060,1,BUY,1.0,111.281250,4d007678-19e2-4116-b852-dff37e8123c1,1334000,183,1746557449648991866,1746557449648991866,NaN,3,2025-05-06 14:50:49.648
1,2025-05-06,19:54:16.538,8092262517946017060,Unknown_8092262517946017060,1,BUY,10.0,111.203125,ed204150-cc58-41c2-9722-86d68dc2522f,1334000,183,1746575656538272312,1746575656538272312,NaN,3,2025-05-06 19:54:16.538
2,2025-05-06,20:17:50.331,8092262517946017060,Unknown_8092262517946017060,2,SELL,4.0,111.265625,43b95a0c-a848-4d5c-aa65-8913b389c594,1334000,183,1746577070331028240,1746577070331028240,NaN,3,2025-05-06 20:17:50.331
3,2025-05-07,04:14:52.340,8092262517946017060,Unknown_8092262517946017060,1,BUY,20.0,111.156250,53121e79-aad5-480e-8f63-64a75365b15b,1334000,183,1746605692340422109,1746605692340422109,NaN,3,2025-05-07 04:14:52.340
4,2025-05-07,05:22:17.623,8092262517946017060,Unknown_8092262517946017060,2,SELL,15.0,111.218750,df6d779c-37c7-4568-af09-b45b686ac654,1334000,183,1746609737623479358,1746609737623479358,NaN,3,2025-05-07 05:22:17.623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2025-06-04,06:54:31.037,14868446001769503902,Unknown_14868446001769503902,2,SELL,15.0,110.468750,8703227e-6bd5-4f36-8bc8-981d7f7c8670,1334000,183,1749034471037746258,1749034471037746258,NaN,2,2025-06-04 06:54:31.037
213,2025-06-04,06:54:31.037,14868446001769503902,Unknown_14868446001769503902,2,SELL,1.0,110.468750,8703227e-6bd5-4f36-8bc8-981d7f7c8670,1334000,183,1749034471037840498,1749034471037840498,NaN,2,2025-06-04 06:54:31.037
214,2025-06-04,06:54:31.037,14868446001769503902,Unknown_14868446001769503902,2,SELL,7.0,110.468750,8703227e-6bd5-4f36-8bc8-981d7f7c8670,1334000,183,1749034471037975136,1749034471037975136,NaN,3,2025-06-04 06:54:31.037
215,2025-06-04,13:00:10.779,14868446001769503902,Unknown_14868446001769503902,1,BUY,10.0,111.187500,8b6f33f8-6263-49f0-abb6-82cde7eef20f,1334000,183,1749056410779188928,1749056410779188928,NaN,3,2025-06-04 13:00:10.779


: 

In [6]:
# Verify P&L claims: TT API vs Local Calculations
import sys
import os
import requests
import pandas as pd

# Add lib directory for TT API imports
sys.path.insert(0, os.path.join(os.getcwd(), 'lib'))

try:
    from trading.tt_api import (
        TTTokenManager, 
        TT_SIM_API_KEY, TT_SIM_API_SECRET,
        APP_NAME, COMPANY_NAME, ENVIRONMENT, TOKEN_FILE
    )
    
    print("=== P&L VERIFICATION: TT API vs Local Data ===")
    
    # 1. Get TT API Position Data
    token_manager = TTTokenManager(
        api_key=TT_SIM_API_KEY,
        api_secret=TT_SIM_API_SECRET,
        app_name=APP_NAME,
        company_name=COMPANY_NAME,
        environment=ENVIRONMENT,
        token_file_base=TOKEN_FILE
    )
    
    url = "https://ttrestapi.trade.tt/ttmonitor/ext_prod_sim/position"
    headers = {
        "x-api-key": token_manager.api_key,
        "accept": "application/json",
        "Authorization": f"Bearer {token_manager.get_token()}"
    }
    params = {"requestId": token_manager.create_request_id()}
    
    response = requests.get(url, headers=headers, params=params, timeout=30)
    tt_data = response.json()
    
    print("\n1. TT API POSITION DATA:")
    total_unrealized_pnl = 0
    total_realized_pnl = 0
    
    for position in tt_data.get('positions', []):
        net_pos = position.get('netPosition', 0)
        unrealized_pnl = position.get('pnl', 0)
        realized_pnl = position.get('realizedPnl', 0)
        instrument_id = position.get('instrumentId', 'Unknown')
        
        print(f"  Instrument {instrument_id}: NetPos={net_pos}, Unrealized=${unrealized_pnl:.2f}, Realized=${realized_pnl:.2f}")
        total_unrealized_pnl += unrealized_pnl
        total_realized_pnl += realized_pnl
    
    print(f"\n  TOTAL from TT API: Unrealized=${total_unrealized_pnl:.2f}, Realized=${total_realized_pnl:.2f}")
    
    # 2. Get Local Fill Data
    print("\n2. LOCAL FILL DATA ANALYSIS:")
    
    # Current ZN Sep25 position from continuous fills
    df_fills = pd.read_csv("data/output/ladder/continuous_fills.csv")
    df_zn_sep25 = df_fills[
        (df_fills["Exchange"] == "CME") & 
        (df_fills["CurrentUser"] == "Eric") & 
        (df_fills["Contract"] == "ZN Sep25")
    ]
    
    buy_qty = df_zn_sep25[df_zn_sep25["SideName"] == "BUY"]["Quantity"].sum()
    sell_qty = df_zn_sep25[df_zn_sep25["SideName"] == "SELL"]["Quantity"].sum()
    local_net_pos = buy_qty - sell_qty
    
    print(f"  ZN Sep25 from local fills: Buy={buy_qty}, Sell={sell_qty}, Net={local_net_pos}")
    
    # Check LIFO state
    try:
        import pickle
        lifo_state = pickle.load(open("lifo_streaming_state.pkl", "rb"))
        lifo_net_pos = sum(stack_item[0] for stack_item in lifo_state['stack'])
        print(f"  LIFO calculated position: {lifo_net_pos}")
    except:
        print("  LIFO state file not found")
    
    # 3. Verification Summary
    print("\n3. VERIFICATION SUMMARY:")
    print("=" * 50)
    
    # Find ZN Sep25 position in TT API data
    zn_sep25_tt_pos = None
    for position in tt_data.get('positions', []):
        if position.get('netPosition') == -1784:  # Known ZN Sep25 position
            zn_sep25_tt_pos = position
            break
    
    if zn_sep25_tt_pos:
        tt_net_pos = zn_sep25_tt_pos.get('netPosition', 0)
        tt_unrealized = zn_sep25_tt_pos.get('pnl', 0)
        tt_realized = zn_sep25_tt_pos.get('realizedPnl', 0)
        
        print(f"TT API ZN Sep25: Net={tt_net_pos}, Unrealized=${tt_unrealized:.2f}, Realized=${tt_realized:.2f}")
        print(f"Local ZN Sep25:  Net={local_net_pos}")
        
        # Position match check
        position_match = "✅ MATCH" if tt_net_pos == local_net_pos else "❌ MISMATCH"
        print(f"Position verification: {position_match}")
        
        # P&L interpretation
        print(f"\nP&L INTERPRETATION:")
        print(f"• Unrealized P&L (${tt_unrealized:.2f}): Current position mark-to-market")
        print(f"• Realized P&L (${tt_realized:.2f}): Only from current ZN Sep25 trades")
        print(f"• Missing historical realized P&L from rolled contracts (Jun25 → Sep25)")
        
    else:
        print("Could not find ZN Sep25 position in TT API data")
    
    print(f"\nCONCLUSION: Both P&L figures from TT API are position-specific, not account-wide")
    
except ImportError as e:
    print(f"Could not import TT API tools: {e}")
    print("Skipping TT API verification")
except Exception as e:
    print(f"Error during verification: {e}")


=== P&L VERIFICATION: TT API vs Local Data ===

1. TT API POSITION DATA:
  Instrument 14868446001769503902: NetPos=55.0, Unrealized=$0.47, Realized=$0.00
  Instrument 1375333834396392980: NetPos=-1784.0, Unrealized=$-237953.12, Realized=$-1015.62

  TOTAL from TT API: Unrealized=$-237952.66, Realized=$-1015.62

2. LOCAL FILL DATA ANALYSIS:
  ZN Sep25 from local fills: Buy=1676.0, Sell=3460.0, Net=-1784.0
  LIFO calculated position: -1784.0

3. VERIFICATION SUMMARY:
TT API ZN Sep25: Net=-1784.0, Unrealized=$-237953.12, Realized=$-1015.62
Local ZN Sep25:  Net=-1784.0
Position verification: ✅ MATCH

P&L INTERPRETATION:
• Unrealized P&L ($-237953.12): Current position mark-to-market
• Realized P&L ($-1015.62): Only from current ZN Sep25 trades
• Missing historical realized P&L from rolled contracts (Jun25 → Sep25)

CONCLUSION: Both P&L figures from TT API are position-specific, not account-wide


In [7]:
# P&L Verification: TT API vs Local Continuous Fills Data
import sys
import os
import requests
import pandas as pd

# Add lib directory for TT API imports
sys.path.insert(0, os.path.join(os.getcwd(), 'lib'))

try:
    # Import TT API components
    from trading.tt_api import (
        TTTokenManager, 
        TT_SIM_API_KEY, TT_SIM_API_SECRET,
        APP_NAME, COMPANY_NAME, ENVIRONMENT, TOKEN_FILE
    )
    
    print("=== P&L VERIFICATION: TT API vs Local Fills ===")
    
    # 1. Get TT API Position Data
    token_manager = TTTokenManager(
        api_key=TT_SIM_API_KEY,
        api_secret=TT_SIM_API_SECRET,
        app_name=APP_NAME,
        company_name=COMPANY_NAME,
        environment=ENVIRONMENT,
        token_file_base=TOKEN_FILE
    )
    
    url = "https://ttrestapi.trade.tt/ttmonitor/ext_prod_sim/position"
    headers = {
        "x-api-key": token_manager.api_key,
        "accept": "application/json",
        "Authorization": f"Bearer {token_manager.get_token()}"
    }
    params = {"requestId": token_manager.create_request_id()}
    
    response = requests.get(url, headers=headers, params=params, timeout=30)
    tt_data = response.json()
    
    print("\n1. TT API CURRENT POSITIONS:")
    tt_total_unrealized = 0
    tt_total_realized = 0
    
    for position in tt_data.get('positions', []):
        net_pos = position.get('netPosition', 0)
        unrealized_pnl = position.get('pnl', 0)
        realized_pnl = position.get('realizedPnl', 0)
        instrument_id = position.get('instrumentId', 'Unknown')
        
        print(f"  Instrument {instrument_id}: NetPos={net_pos}, Unrealized=${unrealized_pnl:.2f}, Realized=${realized_pnl:.2f}")
        tt_total_unrealized += unrealized_pnl
        tt_total_realized += realized_pnl
    
    print(f"\n  TT API TOTALS: Unrealized=${tt_total_unrealized:.2f}, Realized=${tt_total_realized:.2f}")
    
    # 2. Analyze Local Continuous Fills Data
    print("\n2. LOCAL CONTINUOUS FILLS ANALYSIS:")
    print("  NOTE: This CSV contains TRADE DATA only, not calculated P&L")
    
    df_fills = pd.read_csv("data/output/ladder/continuous_fills.csv")
    df_cme = df_fills[df_fills["Exchange"] == "CME"]
    df_eric = df_cme[df_cme["CurrentUser"] == "Eric"]
    
    # ZN Sep25 and ZN Jun25 combined
    df_zn_all = df_eric[df_eric["Contract"].str.contains("ZN", na=False)]
    
    print(f"  Total ZN trades found: {len(df_zn_all)}")
    unique_contracts = list(df_zn_all['Contract'].unique())
    print(f"  Contracts: {unique_contracts}")
    
    # Calculate net position for each contract
    for contract in unique_contracts:
        df_contract = df_zn_all[df_zn_all['Contract'] == contract]
        buy_qty = df_contract[df_contract["SideName"] == "BUY"]["Quantity"].sum()
        sell_qty = df_contract[df_contract["SideName"] == "SELL"]["Quantity"].sum()
        net_pos = buy_qty - sell_qty
        print(f"  {contract}: Buy={buy_qty}, Sell={sell_qty}, Net={net_pos}")
    
    # Total ZN position
    total_buy = df_zn_all[df_zn_all["SideName"] == "BUY"]["Quantity"].sum()
    total_sell = df_zn_all[df_zn_all["SideName"] == "SELL"]["Quantity"].sum()
    total_net = total_buy - total_sell
    
    print(f"\n  TOTAL ZN POSITION: Buy={total_buy}, Sell={total_sell}, Net={total_net}")
    print(f"  ⚠️  To calculate P&L from this data, you would need:")
    print(f"     - LIFO/FIFO matching algorithm (like your existing lifo_pnl_monitor.py)")
    print(f"     - Current market prices for unrealized P&L")
    print(f"     - Contract multipliers (ZN = $1000 per tick)")
    
    # 3. Comparison Summary
    print("\n3. KEY FINDINGS:")
    print("=" * 50)
    
    # Find ZN Sep25 in TT API (the -1784 position)
    zn_sep25_api = None
    for position in tt_data.get('positions', []):
        if position.get('netPosition') == -1784:
            zn_sep25_api = position
            break
    
    if zn_sep25_api:
        api_unrealized = zn_sep25_api.get('pnl', 0)
        api_realized = zn_sep25_api.get('realizedPnl', 0)
        
        print(f"TT API shows only CURRENT positions:")
        print(f"  ZN Sep25: Net=-1784, Unrealized=${api_unrealized:.2f}, Realized=${api_realized:.2f}")
        print(f"\nLocal fills show HISTORICAL trading:")
        print(f"  All ZN contracts combined: Net={total_net}")
        print(f"\nCONCLUSION:")
        print(f"✅ TT API P&L is position-specific (current holdings only)")
        print(f"✅ Historical P&L from rolled contracts (Jun25→Sep25) NOT included")
        print(f"✅ Your TT Desktop's ~$207K realized P&L includes historical trades")
        
    else:
        print("Could not find ZN Sep25 position in TT API data")
        
except ImportError as e:
    print(f"Could not import TT API tools: {e}")
    print("Make sure you're running this from the project root directory")
except Exception as e:
    print(f"Error during verification: {e}")


=== P&L VERIFICATION: TT API vs Local Fills ===

1. TT API CURRENT POSITIONS:
  Instrument 14868446001769503902: NetPos=55.0, Unrealized=$0.47, Realized=$0.00
  Instrument 1375333834396392980: NetPos=-1784.0, Unrealized=$-237953.12, Realized=$-1015.62

  TT API TOTALS: Unrealized=$-237952.66, Realized=$-1015.62

2. LOCAL CONTINUOUS FILLS ANALYSIS:
  Total ZN trades found: 69
  Contracts: ['ZN Sep25']
  ZN Sep25: Buy=1676.0, Sell=3460.0, Net=-1784.0

  TOTAL ZN POSITION: Buy=1676.0, Sell=3460.0, Net=-1784.0

3. KEY FINDINGS:
TT API shows only CURRENT positions:
  ZN Sep25: Net=-1784, Unrealized=$-237953.12, Realized=$-1015.62

Local fills show HISTORICAL trading:
  All ZN contracts combined: Net=-1784.0

CONCLUSION:
✅ TT API P&L is position-specific (current holdings only)
✅ Historical P&L from rolled contracts (Jun25→Sep25) NOT included
✅ Your TT Desktop's ~$207K realized P&L includes historical trades


In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/output/ladder/continuous_fills.csv")
df.head()

,Date,Time,InstrumentId,InstrumentName,Side,SideName,Quantity,Price,OrderId,AccountId,MarketId,TransactTime,TimeStamp,ExecId,OrderStatus,Exchange,Contract,Originator,CurrentUser
0,2025-05-06,14:50:49.648,8092262517946017060,Unknown_8092262517946017060,1,BUY,1.0,111.281250,4d007678-19e2-4116-b852-dff37e8123c1,1334000,183,1746557449648991866,1746557449648991866,NaN,3,CME_Delayed,ZN Jun25,NaN,SPatoliya
1,2025-05-06,19:54:16.538,8092262517946017060,Unknown_8092262517946017060,1,BUY,10.0,111.203125,ed204150-cc58-41c2-9722-86d68dc2522f,1334000,183,1746575656538272312,1746575656538272312,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric
2,2025-05-06,20:17:50.331,8092262517946017060,Unknown_8092262517946017060,2,SELL,4.0,111.265625,43b95a0c-a848-4d5c-aa65-8913b389c594,1334000,183,1746577070331028240,1746577070331028240,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric
3,2025-05-07,04:14:52.340,8092262517946017060,Unknown_8092262517946017060,1,BUY,20.0,111.156250,53121e79-aad5-480e-8f63-64a75365b15b,1334000,183,1746605692340422109,1746605692340422109,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric
4,2025-05-07,05:22:17.623,8092262517946017060,Unknown_8092262517946017060,2,SELL,15.0,111.218750,df6d779c-37c7-4568-af09-b45b686ac654,1334000,183,1746609737623479358,1746609737623479358,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric


In [16]:
import numpy as np

df["qty"] = np.where(df["SideName"] == "BUY", df["Quantity"], -df["Quantity"])

df["datetime"] = df["Date"] + " " + df["Time"]
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values(by="datetime")
df

,Date,Time,InstrumentId,InstrumentName,Side,SideName,Quantity,Price,OrderId,AccountId,...,TransactTime,TimeStamp,ExecId,OrderStatus,Exchange,Contract,Originator,CurrentUser,qty,datetime
0,2025-05-06,14:50:49.648,8092262517946017060,Unknown_8092262517946017060,1,BUY,1.0,111.281250,4d007678-19e2-4116-b852-dff37e8123c1,1334000,...,1746557449648991866,1746557449648991866,NaN,3,CME_Delayed,ZN Jun25,NaN,SPatoliya,1.0,2025-05-06 14:50:49.648
1,2025-05-06,19:54:16.538,8092262517946017060,Unknown_8092262517946017060,1,BUY,10.0,111.203125,ed204150-cc58-41c2-9722-86d68dc2522f,1334000,...,1746575656538272312,1746575656538272312,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,10.0,2025-05-06 19:54:16.538
2,2025-05-06,20:17:50.331,8092262517946017060,Unknown_8092262517946017060,2,SELL,4.0,111.265625,43b95a0c-a848-4d5c-aa65-8913b389c594,1334000,...,1746577070331028240,1746577070331028240,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,-4.0,2025-05-06 20:17:50.331
3,2025-05-07,04:14:52.340,8092262517946017060,Unknown_8092262517946017060,1,BUY,20.0,111.156250,53121e79-aad5-480e-8f63-64a75365b15b,1334000,...,1746605692340422109,1746605692340422109,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,20.0,2025-05-07 04:14:52.340
4,2025-05-07,05:22:17.623,8092262517946017060,Unknown_8092262517946017060,2,SELL,15.0,111.218750,df6d779c-37c7-4568-af09-b45b686ac654,1334000,...,1746609737623479358,1746609737623479358,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,-15.0,2025-05-07 05:22:17.623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2025-07-05,15:45:47.957,1375333834396392980,Unknown_1375333834396392980,1,BUY,807.0,111.390625,50f8c397-8ce3-4306-b10d-ef1abc160dc2,1334000,...,1751744747957825985,1751744747957825985,NaN,3,CME,ZN Sep25,NaN,Eric,807.0,2025-07-05 15:45:47.957
289,2025-07-05,16:15:44.934,1375333834396392980,Unknown_1375333834396392980,2,SELL,1.0,111.343750,37000ea1-43f5-4f76-80b3-cd4cb2fb390f,1334000,...,1751746544934874873,1751746544934874873,NaN,3,CME,ZN Sep25,NaN,Eric,-1.0,2025-07-05 16:15:44.934
290,2025-07-05,16:16:01.379,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,8fd9a8a9-3878-436e-8374-61bb2aa53d8d,1334000,...,1751746561379716794,1751746561379716794,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-05 16:16:01.379
291,2025-07-05,17:44:17.086,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,cdfdc0f1-6a06-4dbf-b6e8-fdac049aeba3,1334000,...,1751751857086012933,1751751857086012933,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-05 17:44:17.086


In [17]:
import pandas as pd
import numpy as np

def pnl(df1):
    df = df1.copy()

    df["qty"] = np.where(df["SideName"] == "BUY", df["Quantity"], -df["Quantity"])
    
    df["cum_qty"] = df["qty"].cumsum()

    df["delta_price"] = df["Price"] - df["Price"].shift(-1)   

    df["cumm_pnl"] = (df["delta_price"] * df["cum_qty"]*62.5*16).cumsum()    
    
    return df


In [23]:
pnl(df[(df["Contract"]=="ZN Sep25") & (df["Exchange"]=="CME") & (df['datetime']>='2025-07-03 17:38:00') & (df['datetime']<='2025-07-05 17:38:00')])

,Date,Time,InstrumentId,InstrumentName,Side,SideName,Quantity,Price,OrderId,AccountId,...,OrderStatus,Exchange,Contract,Originator,CurrentUser,qty,datetime,cum_qty,delta_price,cumm_pnl
273,2025-07-03,17:41:00.006,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.265625,1ce3510b-4347-4707-8b52-e283b11b9e13,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-03 17:41:00.006,1.0,0.000000,0.000
274,2025-07-03,17:41:57.338,1375333834396392980,Unknown_1375333834396392980,1,BUY,2.0,111.265625,f5368272-922e-4bee-8197-1ff0e433b2fb,1334000,...,3,CME,ZN Sep25,NaN,Eric,2.0,2025-07-03 17:41:57.338,3.0,0.031250,93.750
275,2025-07-03,18:27:01.561,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.234375,33f120f9-1c34-4508-a651-7a8172775e42,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-03 18:27:01.561,4.0,0.000000,93.750
276,2025-07-03,18:48:36.525,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.234375,e4716b30-fc6d-4269-8d94-7e99a6c7faa0,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-03 18:48:36.525,5.0,0.015625,171.875
277,2025-07-03,19:27:22.691,1375333834396392980,Unknown_1375333834396392980,2,SELL,727.0,111.218750,e26cf880-c539-434f-ac5a-263372fedb87,1334000,...,2,CME,ZN Sep25,NaN,Eric,-727.0,2025-07-03 19:27:22.691,-722.0,0.000000,171.875
278,2025-07-03,19:27:24.234,1375333834396392980,Unknown_1375333834396392980,2,SELL,44.0,111.218750,e26cf880-c539-434f-ac5a-263372fedb87,1334000,...,2,CME,ZN Sep25,NaN,Eric,-44.0,2025-07-03 19:27:24.234,-766.0,0.000000,171.875
279,2025-07-03,19:27:24.326,1375333834396392980,Unknown_1375333834396392980,2,SELL,1.0,111.218750,e26cf880-c539-434f-ac5a-263372fedb87,1334000,...,2,CME,ZN Sep25,NaN,Eric,-1.0,2025-07-03 19:27:24.326,-767.0,-0.140625,108031.250
280,2025-07-04,11:08:26.706,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,45d16498-4501-4d54-8d85-7ce1c03dc3af,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-04 11:08:26.706,-766.0,0.000000,108031.250
281,2025-07-04,15:22:58.723,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,27e299cb-245c-4701-9d1e-661649471b67,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-04 15:22:58.723,-765.0,0.000000,108031.250
282,2025-07-04,15:23:23.349,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,41ccb8b3-e9ab-4421-a5fe-6d18c78d6df6,1334000,...,3,CME,ZN Sep25,NaN,Eric,1.0,2025-07-04 15:23:23.349,-764.0,0.000000,108031.250


In [44]:
111 + 12.5/32

111.390625

In [38]:
-442453.125-(1784*62.55*(111 + (11.5/32) -111.359375)*16)

-442453.125

In [17]:
df

,Date,Time,InstrumentId,InstrumentName,Side,SideName,Quantity,Price,OrderId,AccountId,...,ExecId,OrderStatus,Exchange,Contract,Originator,CurrentUser,qty,cum_qty,delta_price,cumm_pnl
0,2025-05-06,14:50:49.648,8092262517946017060,Unknown_8092262517946017060,1,BUY,1.0,111.281250,4d007678-19e2-4116-b852-dff37e8123c1,1334000,...,NaN,3,CME_Delayed,ZN Jun25,NaN,SPatoliya,1.0,1.0,0.078125,4.882812e+00
1,2025-05-06,19:54:16.538,8092262517946017060,Unknown_8092262517946017060,1,BUY,10.0,111.203125,ed204150-cc58-41c2-9722-86d68dc2522f,1334000,...,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,10.0,11.0,-0.062500,-3.808594e+01
2,2025-05-06,20:17:50.331,8092262517946017060,Unknown_8092262517946017060,2,SELL,4.0,111.265625,43b95a0c-a848-4d5c-aa65-8913b389c594,1334000,...,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,-4.0,7.0,0.109375,9.765625e+00
3,2025-05-07,04:14:52.340,8092262517946017060,Unknown_8092262517946017060,1,BUY,20.0,111.156250,53121e79-aad5-480e-8f63-64a75365b15b,1334000,...,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,20.0,27.0,-0.062500,-9.570312e+01
4,2025-05-07,05:22:17.623,8092262517946017060,Unknown_8092262517946017060,2,SELL,15.0,111.218750,df6d779c-37c7-4568-af09-b45b686ac654,1334000,...,NaN,3,CME_Delayed,ZN Jun25,NaN,Eric,-15.0,12.0,-0.328125,-3.417969e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,2025-07-04,15:22:58.723,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,27e299cb-245c-4701-9d1e-661649471b67,1334000,...,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,-916.0,0.000000,-1.864721e+08
282,2025-07-04,15:23:23.349,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,41ccb8b3-e9ab-4421-a5fe-6d18c78d6df6,1334000,...,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,-915.0,0.000000,-1.864721e+08
283,2025-07-04,15:24:53.725,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,d4d9ed30-fed1-4dac-a853-4152928a9043,1334000,...,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,-914.0,0.000000,-1.864721e+08
284,2025-07-04,15:25:02.649,1375333834396392980,Unknown_1375333834396392980,1,BUY,1.0,111.359375,ae8d5299-6533-4a02-8cf4-326e99676aa0,1334000,...,NaN,3,CME,ZN Sep25,NaN,Eric,1.0,-913.0,0.000000,-1.864721e+08


In [18]:
-1.864721e+08

-186472100.0

In [40]:
df = df[(df["Contract"]=="ZN Sep25") & (df["Exchange"]=="CME")]

In [41]:
# ---------- PARAMETERS ----------
POINT_VALUE = 1_000          # $ value of ONE full price-point for the contract
mark_px     = df["Price"].iloc[-1]   # use last traded price as the mark

# ---------- 1 · build signed quantity ----------
df["signed_qty"] = np.where(df["SideName"] == "BUY",
                            df["Quantity"],
                            -df["Quantity"])

# ---------- 2 · headline P/L (realised + open) ----------
df["trade_pnl_$"] = (mark_px - df["Price"]) * df["signed_qty"] * POINT_VALUE
total_pnl = df["trade_pnl_$"].sum()

# ---------- 3 · net position & average-open price ----------
net_pos = df["signed_qty"].sum()

if net_pos == 0:                       # totally flat → all P/L is realised
    open_pnl  = 0.0
    realised  = total_pnl
    avg_open  = None
else:
    # figure out WHICH lots are still open so we can get their avg cost
    remaining = abs(net_pos)
    direction = 1 if net_pos > 0 else -1          # long = +1, short = -1
    value_sum = 0.0                              # Σ(price * qty) for open lots

    # walk **backwards** through fills until we've accounted for the open qty
    for price, qty in zip(df["Price"][::-1], df["signed_qty"][::-1]):
        if qty * direction <= 0:                 # opposite-side trade (already closed)
            continue
        take = min(abs(qty), remaining)
        value_sum += price * take
        remaining -= take
        if remaining == 0:
            break

    avg_open = value_sum / abs(net_pos)          # average entry price of the open lot(s)
    open_pnl = (mark_px - avg_open) * net_pos * POINT_VALUE
    realised = total_pnl - open_pnl

# ---------- 4 · results ----------
print(f"Mark price             : {mark_px:.6f}")
print(f"Net position (contracts): {net_pos:+}")
if avg_open is not None:
    print(f"Average open price     : {avg_open:.6f}")
print()
print(f"P/L Open   (un-realised): ${open_pnl:,.2f}")
print(f"P/L Realised            : ${realised:,.2f}")
print(f"----------------------------------------------")
print(f"TOTAL P/L               : ${total_pnl:,.2f}")

Mark price             : 111.359375
Net position (contracts): -1784.0
Average open price     : 111.512656

P/L Open   (un-realised): $273,453.12
P/L Realised            : $169,000.00
----------------------------------------------
TOTAL P/L               : $442,453.12


C:\Users\YamanSanghavi\AppData\Local\Temp\ipykernel_36616\3927073035.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["signed_qty"] = np.where(df["SideName"] == "BUY",
C:\Users\YamanSanghavi\AppData\Local\Temp\ipykernel_36616\3927073035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["trade_pnl_$"] = (mark_px - df["Price"]) * df["signed_qty"] * POINT_VALUE
